<a href="https://colab.research.google.com/github/ruebot/notebooks/blob/main/yfile_indigenous_mallet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cribbed from https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/09-Topic-Modeling-Without-Mallet.html

In [1]:
%%capture

import os
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  !java -version   
install_java()

!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

In [2]:
%%capture
!pip install tomotopy
!pip install little_mallet_wrapper
!pip install seaborn
!pip install git+https://github.com/maria-antoniak/little-mallet-wrapper.git

In [3]:
import tomotopy as tp
import little_mallet_wrapper
import seaborn
import glob
from pathlib import Path

os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
path_to_mallet = '/content/mallet-2.0.8/bin/mallet'

In [11]:
%%capture
!wget https://www.dropbox.com/s/q261fcbal6ltpmd/corpus.zip
!unzip corpus.zip

In [12]:
files = glob.glob(f"corpus/*.txt")

In [13]:
training_data = []
original_texts = []
titles = []

for file in files:
    text = open(file, encoding='utf-8').read()
    processed_text = little_mallet_wrapper.process_string(text, numbers='remove')
    training_data.append(processed_text)
    original_texts.append(text)
    titles.append(Path(file).stem)

In [14]:
len(training_data), len(original_texts), len(titles)

(599, 599, 599)

In [15]:
# Number of topics to return
num_topics = 25
# Numer of topic words to print out
num_topic_words = 10

# Intialize the model
model = tp.LDAModel(k=num_topics)

# Add each document to the model, after splitting it up into words
for text in training_data:
    model.add_doc(text.strip().split())
    
print("Topic Model Training...\n\n")
# Iterate over the data 10 times
iterations = 10
for i in range(0, 100, iterations):
    model.train(iterations)
    print(f'Iteration: {i}\tLog-likelihood: {model.ll_per_word}')

print("\nTopic Model Results:\n\n")
# Print out top 10 words for each topic
topics = []
topic_individual_words = []
for topic_number in range(0, num_topics):
    topic_words = ' '.join(word for word, prob in model.get_topic_words(topic_id=topic_number, top_n=num_topic_words))
    topics.append(topic_words)
    topic_individual_words.append(topic_words.split())
    print(f"✨Topic {topic_number}✨\n\n{topic_words}\n")

Topic Model Training...


Iteration: 0	Log-likelihood: -9.710518060659242
Iteration: 10	Log-likelihood: -9.348851495100144
Iteration: 20	Log-likelihood: -9.20793703640172
Iteration: 30	Log-likelihood: -9.124508405219386
Iteration: 40	Log-likelihood: -9.077592273216881
Iteration: 50	Log-likelihood: -9.038769602074353
Iteration: 60	Log-likelihood: -9.005978206994438
Iteration: 70	Log-likelihood: -8.981096639644207
Iteration: 80	Log-likelihood: -8.959330651993042
Iteration: 90	Log-likelihood: -8.941171540869826

Topic Model Results:


✨Topic 0✨

university york professor story quoted read full mentioned aug news

✨Topic 1✨

york university students student faculty community also program black president

✨Topic 2✨

award program school year graduate awards faculty career director excellence

✨Topic 3✨

says people said like one would way many think also

✨Topic 4✨

may business reported star jan year schulich mail north week

✨Topic 5✨

aboriginal students event place series college confer

In [16]:
topic_distributions = [list(doc.get_topic_dist()) for doc in model.docs]

In [17]:
from IPython.display import Markdown, display
import re

def make_md(string):
    display(Markdown(str(string)))

def get_top_docs(docs, topic_distributions, topic_index, n=5):
    
    sorted_data = sorted([(_distribution[topic_index], _document) 
                          for _distribution, _document 
                          in zip(topic_distributions, docs)], reverse=True)
    
    topic_words = topics[topic_index]
    
    make_md(f"### ✨Topic {topic_index}✨\n\n{topic_words}\n\n---")
    
    for probability, doc in sorted_data[:n]:
        # Make topic words bolded
        for word in topic_words.split():
            if word in doc.lower():
                doc = re.sub(f"\\b{word}\\b", f"**{word}**", doc, re.IGNORECASE)
        
        make_md(f'✨  \n**Topic Probability**: {probability}  \n**Document**: {doc}\n\n')
    
    return

In [18]:
get_top_docs(titles, topic_distributions, topic_index=0, n=6)

### ✨Topic 0✨

university york professor story quoted read full mentioned aug news

---

✨  
**Topic Probability**: 0.6597138047218323  
**Document**: 2021-11-09-**york**-u-in-the-**news**-daylight-savings-covid-19-herd-immunity-and-more



✨  
**Topic Probability**: 0.6590621471405029  
**Document**: 2022-08-24-**york**-u-in-the-**news**-chinese-**news**-outlets-emergency-preparedness-and-more



✨  
**Topic Probability**: 0.6398530006408691  
**Document**: 2022-08-03-**york**-u-in-the-**news**-pope-sleepwalking-and-lakes



✨  
**Topic Probability**: 0.6370700597763062  
**Document**: 2022-09-29-**york**-u-in-the-**news**-high-food-prices-skinai-and-more



✨  
**Topic Probability**: 0.6307199597358704  
**Document**: 2022-08-17-**york**-u-in-the-**news**-refugees-stem-team-and-more



✨  
**Topic Probability**: 0.6306686997413635  
**Document**: 2021-10-05-**york**-u-in-the-**news**-individual-education-plans-homelessness-and-more



In [19]:
get_top_docs(titles, topic_distributions, topic_index=1, n=5)

### ✨Topic 1✨

york university students student faculty community also program black president

---

✨  
**Topic Probability**: 0.44274115562438965  
**Document**: 2022-05-29-**faculty**-**students**-administrators-encouraged-to-share-**york**-experiences-in-macleans-survey



✨  
**Topic Probability**: 0.43585604429244995  
**Document**: 2010-01-22-**york**-awarded-400000-to-support-aboriginal-learners



✨  
**Topic Probability**: 0.4268280863761902  
**Document**: 2021-10-17-**york**-improves-in-several-categories-in-macleans-2022-**university**-rankings



✨  
**Topic Probability**: 0.38662272691726685  
**Document**: 2022-05-18-alectra-and-euc-establish-alectra-equity-diversity-and-inclusion-undergraduate-awards



✨  
**Topic Probability**: 0.38086599111557007  
**Document**: 2022-01-17-**york**-hosts-events-to-mark-bell-lets-talk-campaign



In [20]:
get_top_docs(original_texts, topic_distributions, topic_index=1, n=5)

### ✨Topic 1✨

york university students student faculty community also program black president

---

✨  
**Topic Probability**: 0.44274115562438965  
**Document**: Each year, thousands of prospective **university** **students** and their parents turn to Maclean’s annual University Rankings for information to help them select a **university** that suits their needs.
The Maclean’s University Rankings is a comprehensive publication that classifies each Canadian university on a broad range of criteria with responses and data gathered from two annual surveys. The Canadian news magazine conducts the surveys to gather the opinions of university **faculty** and senior administrators, undergraduate **students** and businesspeople from across Canada. The rankings are the most influential within Canada for students, employers and others considering Canadian universities.
The 2023 Reputational Survey is now available for qualified **faculty** and senior administrators to complete and will be open until September 2022. The rankings factor in responses from the 2023 Student Survey, which will **also** be available until September 2022. It polls undergraduate students about their university experience and satisfaction with teaching and learning, mental health services, sexual violence prevention, academic advising, **student** and residence life.
The Maclean’s Reputational Survey for Faculty (English version) is available at: https://www.surveymonkey.com/r/MacleansAcademicSurvey202202.
The French-language version of the Reputational Survey is available at: https://fr.surveymonkey.com/r/MacleansSondageEnseignant202202.
The Maclean’s Student Survey (English version) is available at: https://www.surveymonkey.com/r/MacleansStudents202202.
The French-language version of the Student Survey is available at: https://fr.surveymonkey.com/r/MacleansEtudiants202201.
“The Maclean’s University Rankings is an important guide for prospective students who are deciding on where to attend university, and for others looking to assess potential future employees or university partners. It **also** provides important feedback to the University on what we are known for doing well and where there is room for future reputational improvement. I encourage faculty and senior administrators to complete the Reputational Survey to help highlight the groundbreaking work the University is doing and the clear advantages to choosing York,” says Lisa Philipps, provost and vice-**president** academic.
Individuals are drawn to York University because they are inspired to drive positive change. York University is taking bold steps to right the future:
“York’s scholars and researchers are pioneering new ideas and discoveries that are not only enhancing the reputation of our institution but will have a positive change on the communities we serve,” says Amir Asif, vice-**president**, research and innovation. “York is home to Canada’s brightest minds in mathematical modelling for COVID-19, and in fact, our researchers have been instrumental in collaborating with national and international leaders to advise on the trajectory of COVID-19 pandemic in Canada and other parts of the world, including Africa. This will continue to be incredibly important for managing future pandemics and health crises.”
From a research perspective, York University has much to be proud of:
In the Maclean’s 2022 University Rankings released in the Fall of 2021, York University was well recognized for advancing its vision to provide a high-quality education at a research-intensive university informed by a clear set of core values. The rankings highlight solid progress in several major areas, including an impressive sixth place ranking against all other comprehensive universities in Canada for the second consecutive year.
York’s rankings were also up in key priority areas, including the national Reputation category, where the University climbed two spots to 18 out of 49. The Reputation score is comprised of the following three categories, and York showed improvements over the last year in all of them:
The University **community** is tackling complex global issues and preparing students for long-term success. This commitment is reflected in the Student Satisfaction section (Comprehensive University category), where there were noteworthy achievements in the 2022 rankings, with York up two spots in the Mental Health Services category, up three spots in Promoting Indigenous Identity and up five spots in Steps to Prevent Sexual Assault.




✨  
**Topic Probability**: 0.43585604429244995  
**Document**: The Ministry of Training, Colleges & Universities (MTCU) announced Wednesday that York University will receive $400,000 for the 2009-2010 year to develop and deliver support services to ensure positive outcomes for aboriginal learners and to support institutional development. The ministry **also** held out the prospect of further funding in the following two years, based on results.
The plan is to enhance and expand services for current aboriginal **students**, including peer mentoring, laptop lending, writing support and **student** leadership; to attract more aboriginal **students** to York through high-school visits by an aboriginal recruiter and to work with high school aboriginal counsellors and advisers; to perform **community** outreach; to develop bridging programs working with service agencies to improve access to York; and to establish an Aboriginal Alumni Association.
Left: Lisa Odjig, world champion hoop dancer performing at a powwow held at York
“These funds will allow York to greatly enhance the support services to the aboriginal **student** population on campus and to develop the necessary resources in place to allow this expansion, such as additional staff and space,” says Randy Pitawanakwat, coordinator of Aboriginal Student Community in the Centre for Student Community & Leadership Development.
York plans to use the funds to enhance its youth engagement strategy, such as the Riverdale Project which started in September and gives aboriginal students from Riverdale Collegiate Institute the opportunity to take a York University course and receive a credit toward future studies. 
The successful recruitment of aboriginal learners often hinges on the involvement of agencies and **community** centres, as well as schools, and this is another area York will concentrate on developing. In addition, as campus visits and viewbooks have been shown to be more influential with aboriginal students than for the non-aboriginal population, York will make more use of those strategies. 
Already, there are a growing number of York-affiliated people and programs working in aboriginal communities and with aboriginal agencies. There are **also** many representatives of aboriginal communities and agencies who attend cultural events at York – especially the annual powwow – and who give their time and expertise to enrich the experience of aboriginal learners. 
These connections attract students, help retain them and add to their satisfaction and success, says Pitawanakwat. 
Right: Derrick Bressette, head male dancer at the annual pow wow held at York
The goal is to provide continued funding for an aboriginal recruitment officer and other staffing positions that would allow for further aboriginal initiatives at York, such as strengthening community ties, promoting York to potential students and being present in the aboriginal community in a significantly improved way. 
More than 60 per cent of Ontario’s aboriginal population lives in urban areas. Thirty-two per cent lives in Central Ontario, which includes the Greater Toronto Area and where 86 per cent of York’s aboriginal applicants come from. The majority of them self-identify as North American Indian. 
Over the next three years, York will increase the number of aboriginal students applying to and attending the University. In addition to improving access and augmenting its links with the aboriginal community, York hopes to enhance the support services available to aboriginal learners and improve the retention and graduation rates of aboriginal students. As well, a more robust aboriginal community presence will be developed on the campus. 
To help achieve these goals, York will look at enhancing, improving and developing space for aboriginal students and developing a Web strategy, such as computer literacy programs targeted to meet the needs of aboriginal populations. York also hopes to improve cultural programs, including elders-in-residence, an indigenous speaker series and the Aboriginal Awareness Days and powwow event. Some of the other goals include enhancing aboriginal student leadership and establishing aboriginal student peer health education, among other things.
The MTCU’s new consolidated funding approach will use a results-based process where the ministry will track the progress in improving postsecondary education and training outcomes for aboriginal learners. That could mean an additional $400,000 for each of the two subsequent school years – 2010-2011 and 2011-2012.
The approach builds on the Aboriginal Education & Training Strategy, as well as investments made through the Reaching Higher Plan’s Access to Opportunities Strategy (aboriginal-specific component), and other special purpose grants targeted to aboriginal postsecondary education initiatives. 
For more information, contact Randy Pitawanakwat, coordinator of Aboriginal Student Community in the Centre for Student Community & Leadership Development, at ext. 22607 or rpitawan@yorku.ca.




✨  
**Topic Probability**: 0.4268280863761902  
**Document**: As captured in the newly released Maclean’s 2022 University Rankings, York University has been successful in advancing our vision to provide a high-quality education at a research-intensive **university** informed by a clear set of core values. The rankings highlight solid progress in several important areas, including an impressive sixth-place ranking against all other comprehensive universities in Canada for the second consecutive year. 
Despite the complex challenges presented by the global pandemic, the rankings reflect York’s sharp focus on empowering our **community** to build a better future. Guided by the University Academic Plan 2020-25, York is on a path for continued success in strengthening our impact on the United Nations Sustainable Development Goals and advancing our strategic priorities – 21st-century learning, knowledge for the future, from access to success, global engagement, working in partnership and living well together.
“York’s sixth-place ranking as one of Canada’s top comprehensive schools confirms to our **students**, and the broader **community**, that York is providing the high-quality learning experience and research needed to produce the talent and innovation that will keep Canada competitive,” said President and Vice-Chancellor Rhonda L. Lenton. “Over the past year and a half, York has demonstrated that our reputation for relevant programming, critical inquiry, scholarship, knowledge translation and internationalization extends well beyond our campus walls.”
York’s rankings are up in key priority areas, including the national Reputation category, where we climbed two spots to 18 out of 49. The Reputation score is comprised of the following three categories, and York showed improvements over last year in all of them:
As a community, we are tackling complex global issues and preparing **students** for long-term success. This commitment is reflected in the Student Satisfaction section (Comprehensive University category), where there are noteworthy achievements, with York up two spots in the Mental Health Services category, up three spots in Promoting Indigenous Identity and up five spots in Steps to Prevent Sexual Assault.
Individuals drawn to York come inspired to drive positive change and are inspired by each other to do more. Many of our students pursue opportunities to strengthen their impact in their local communities and around the world, facilitated by the entrepreneurial activities and experiential education available in their programs. Looking forward, the University will continue to meet the changing needs of our students and of society through leading scholarship and research, creative new programming, and flexible learning options offered across our multiple campuses and in collaboration with partners in the private and public sectors. Together, we will relentlessly pursue our shared commitment to enhance the well-being of the communities we serve. 
There is much to be proud of and – importantly – more work to do.




✨  
**Topic Probability**: 0.38662272691726685  
**Document**: “We are proud to have Alectra as one of our partners and are very grateful for their sustained and active involvement with the York University **community**, especially in support of **students**, research and transformational initiatives,” says EUC Dean Alice Hovorka. “The Alectra EDI Undergraduate Awards will empower **students** to become champions of justice and sustainability – and will increase equitable access to post-secondary education. In EUC, we are committed to providing accessible education, embracing diversity and empowering all students as changemakers in their own lives and in the lives of others. The Alectra EDI Undergraduate Awards will ensure sustainable pathways and financial support to historically underrepresented students.”
Alectra will contribute $84,000 (over four years) to support two domestic undergraduate Black and/or Indigenous students with full tuition, a flex fund, textbooks and supplies for their full-time degree at York University’s Faculty of Environmental and Urban Change. The Alectra EDI Undergraduate Awards will create opportunities for Black and Indigenous students and drive positive change.
“At Alectra, we believe that making positive change requires dynamic and diverse thinking,” says Brian Bentz, **president** and CEO, Alectra Inc. “We’re excited to support the efforts of York University’s Faculty of Environmental and Urban Change to expand opportunities for under-represented students to thrive in a **program** devoted to addressing the serious urban and environmental challenges that are facing us today.”
In 2020, Alectra began to launch a series of broad-based EDI initiatives and has supported local anti-racism programs delivered through activism, **community** building and education. The partnership with EUC is the first undergraduate scholarship **program** for racialized students that Alectra has sponsored.




✨  
**Topic Probability**: 0.38086599111557007  
**Document**: Members of the York University **community** are invited to join Athletics & Recreation for a number of events during the Bell Let’s Talk campaign, which runs virtually at York’s campuses from Jan. 17 to 27. The University will host events that are free and open to **students**, staff and **faculty**. Bell Let’s Talk events help end the stigma surrounding mental illness by sharing thoughts, experiences and support through virtual events and on social media.
Monday, Jan. 17Mental Health 101 – Navigating Mental Health Resources on Campus – open to students12:30 to 1:30 p.m.Zoom link: https://yorku.zoom.us/meeting/register/tJEldu6ppjojG9AE3aBK8tH14dd0cdzPtf8HThe quality of your health and well-being impacts everything from managing your stress, making time for your loved ones, and still working towards your academic success. Join the Peer Health Educator Team to learn how to:
Tuesday, Jan. 18LGBTQ2S+ Student Support Group – open to students1 to 2 p.m.Register: To register for this virtual session, email Massimo Zangari from Student Counselling, Health & Well-being (SCHW) at mass23@yorku.caSCHW’s LGBTQ2S+ counsellors provide a safe and positive space where stories can be shared, support given, and connections can be made. Group members are invited to bring in topics to discuss – this group is a space to share as much or as little as you want. So, drop in – no registration required. 
Wednesday, Jan. 19Mental Health & Well-being Seminar – open to students12:30 to 1:45 p.m.Zoom link: https://yorku.zoom.us/meeting/register/tJIkcemrrzgoHNPy9UtzmIe44VO9_3agajWbContact: Shailak@yorku.ca                                                                                                                                               Transitioning to **university** is challenging enough during the best of times. As we navigate the hybrid learning environment, many of us are coping with “change fatigue” as we continue to adapt to the uncertainty of the pandemic. The quality of our well-being impacts everything from academic success, physical health, our ability to manage stressors and our relationships with loved ones.Join the Peer Health Educator Team as we:
Thursday, Jan. 20Nutrition Habits for Success – open to all3 to 3:45 p.m.Zoom link: https://yorku.zoom.us/j/95975866935Join Athletics & Recreation’s Nutrition Coach for this 45-minute session, as we focus on easy nutrition tips and habits to support better mental and physical well-being.
Friday, Jan. 21keep.meSAFE Virtual Booth for International Students and Students Studying Out of Province – open to students11:30 a.m. to 12:30 p.m.Zoom link: https://us06web.zoom.us/j/84210804215This year’s Bell Let’s Talk Day campaign highlights the ways that we can support ourselves and those we care about through actions like listening, being there and talking. Join the keep.meSAFE services at their virtual booth where you can get information on mental health resources and supports available to you as international **students** and students studying out of province. No registration required.
Friday, Jan. 21MUV Yoga – open to staff, **faculty** and instructorsNoon to 12:30 p.m.Zoom link: https://yorku.zoom.us/j/93770192798Session offered by Athletics & Recreation Class description: Accessible for all levels. This class will guide you through rhythmic vinyasa flow with a strong focus on breathing techniques to energize your practice. This is followed by a meditative yin yoga practice allowing for longer held postures to promote recovery in both mind and body.
Friday, Jan. 21 MUV Yoga Retreat – open to students5 to 6 p.m.Zoom link: https://yorku.zoom.us/j/98141928860Session offered by Athletics & RecreationClass description: Integral yoga for the mind and body. Yoga retreat promotes holistic health, de-stress, and well-being by strengthening, increasing endurance, and stabilizing the mind, body and breathing. Move towards wellness in your own unique way through postures, breathing exercises, and a wide range of meditations.
Monday, Jan. 24Sleep for Better Grades – open to students12:30 to 1:30 p.m.Zoom link: https://yorku.zoom.us/meeting/register/tJIrd-6rrjwqHt2U2fMsKNIjDKRNdTEsj6-LWhen it comes to the topic of sleep, it seems we can’t get enough. In our Sleep 101 series, we explored the tried and tested tips to get your sleep back on track and improve your sleep hygiene to support better grades and academic success. This workshop will explore the effects of these all-nighters on our minds, bodies and how we can both prioritize our sleep and find practices to optimize whatever amount of sleep we are getting.
Tuesday, Jan. 25Bell Let’s Talk BIPOC Speaker Session with Larissa CrawfordNoon to 1 p.m. (presentation and Q-and-A open to all)1 to 1:30 p.m. (session debrief open to BIPOC students)Zoom link: TBAThis conversation will explore concrete practices and skills that can support students in navigating the mental health impacts of racism and ableism. York University alumna Larissa Crawford will hold space for all students in her presentation and Q-and-A. Following a short break, space for Black, Indigenous, and racialized students will be held in a session debrief.
Tuesday, Jan. 25 Employee & Family Assistance Program (EFAP) Overview – open to staff, faculty and instructors Noon to 12:40 p.m. To register for this virtual session, click here.   The Employee and Family Assistance Program (EFAP) is a confidential and voluntary support service to help you with solving issues related to personal life and or workplace-related issues. Whether the problem is personal such as child/elder care, adjusting to job responsibilities or other issues connected to the workplace, EFAP can be a helpful service to address these challenges through a full range of support services. This session is designed to provide an overview of the programs available to staff, faculty and instructors.
Tuesday, Jan. 25 Glendon Student Mental Health Town Hall – open to students2 to 3:30 p.m. Registration link: https://yorku.zoom.us/meeting/register/tJcsfumgqjkpGdbHtqUoKI3rGq7_tGqo4NymThe Office of Student Affairs, in partnership with the Glendon College Student Union and the Accessibility, Well-Being and Counselling Centre, is inviting all **community** members to join this live town hall in a safe space to speak openly and honestly about what is on their mind. Experts from the Counselling Centre will **also** present resources and offer tools to support the mental health of our students. Students are welcome to send in their questions, as well as preferences regarding topics they’d like covered here.Note that the session will be delivered in English, questions and clarifications can be requested in French. Wednesday, Jan. 26 – BELL LET’S TALK DAY!Building Meaningful Connections in an Online World – open to allNoon to 1 p.m. Zoom link: https://yorku.zoom.us/meeting/register/tJ0sdu2uqT0vGNcE_3NJAdN1wtZw5jTMB_4NThe Faculty of Graduate Studies (FGS) is committed to promoting a healthy, inclusive and supportive environment that fosters mental health and well-being among members of our community. In this session, we will discuss what our personal understanding is of a “meaningful connection,” explore the current research related to the COVID-19 pandemic and the ways in which it has impacted how we relate to one another and the small (but important) steps we can take to cultivate and nurture a sense of belonging with our communities and/or supports. In partnership with Bell Let’s Talk Day, join FGS as we discuss building meaningful connections in an online world. This virtual event is open to all students, staff, faculty, instructors and alumni of York University.
Wednesday, Jan. 26Mental Health 201 – Supporting Peers in Distress – open to students12:30 to 2 p.m. Zoom link: https://yorku.zoom.us/meeting/register/tJ0vdO-trTwvHtMvl2pHQlzbSVyB-yg7sT2QMental Health 201 aims to provide students with the skills needed to recognize, respond, and refer other peers with mental health concerns.The learning objectives of this session include:
Trans and Non-Binary Support Group – open to students4 to 5p.m. Register: To register for this virtual session, email Massimo Zangari from SCHW at mass23@yorku.caJoin Student Counselling, Health & Well-being’s (SCHW’s) monthly online group to meet other York students navigating gender in a relaxed and safe space. Our group is facilitated by Jay Jonah, a trans-masculine social worker who specializes in supporting trans and non-binary folxs. This group is a space to share as much or as little as you want.  Ask questions, just listen in, or chat and make some new friends! Open to all trans, non-binary, gender-non-conforming folxs, and those who are questioning their gender.Schulich – Start Up Event – open to students7 to 8:30 p.m. Zoom link: https://yorku.zoom.us/meeting/register/tJIufuqtpjguGNQaaiSqIhgdbAIPFOd6kbfdJoin us as Luke Vigeant, **president** of Inkblot chats with Entrepreneur-In-Residence and Founder of #REALTALK, Cherry Rose Tan. Learn from these two powerhouse mental health advocates as they lead a crucial discussion on founder mental health in Canada. Inkblot is a mental health service based in Toronto that matches you with certified therapists and coaches based on your needs and preferences. This pairs perfectly with #REALTALK and their mission to empower every leader to learn, connect, and flourish in the tech industry through difficult discussions usually centred around mental health. Both companies have been featured by various news and media outlets such as Forbes, CBC, The Globe and Mail, Global News, the Huffington Post and more. 
Thursday, Jan. 27Employee & Family Assistance Program (EFAP) Overview – open to staff, faculty and instructorsNoon to 12:40 p.m. Register: for this virtual session, click here.  The Employee and Family Assistance Program (EFAP) is a confidential and voluntary support service to help you with solving issues related to personal life and or work-related issues. Whether the problem is personal such as child/elder care, adjusting to job responsibilities or other issues connected to the workplace, EFAP can be a helpful service to address these challenges through a full range of support services. This session is designed to provide an overview of the programs available to staff, faculty and instructors.


